In [1]:
import os
import warnings
import numpy as np
import pandas as pd
import random

warnings.filterwarnings('ignore')
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

In [2]:
import numpy as np

BATCH_SIZE = 30
EPOCHS = 100
SEED = 2

In [3]:
twitter_data_pos = pd.read_csv("processedPositive.csv")
#twitter_data_neu = pd.read_csv("processedNeutral.csv")
twitter_data_neg = pd.read_csv("processedNegative.csv")

all_data = pd.DataFrame()

pos_tweets = twitter_data_pos.columns
num_pos = pos_tweets.size
#neu_tweets = twitter_data_neu.columns
#num_neu = neu_tweets.size
neg_tweets = twitter_data_neg.columns
num_neg = neg_tweets.size

all_tweets = pos_tweets.append(neg_tweets)
#all_tweets = all_tweets.append(neu_tweets)
values = [1 for i in range(num_pos)] + [0 for i in range(num_neg)]
#values = values + [0 for i in range(num_neu)]

all_data["Tweets"] = all_tweets
all_data["Score"] = values
all_data.head()

,Tweets,Score
0,An inspiration in all aspects: Fashion,1
1,fitness,1
2,beauty and personality. :)KISSES TheFashionIcon,1
3,Apka Apna Awam Ka Channel Frankline Tv Aam Adm...,1
4,Beautiful album from the greatest unsung guit...,1


In [4]:
def less_than_8(string):
    if len(string.split()) < 8:
        return True
    else:
        return False
less_than_8("I am going to the park today and eating food")

False

In [5]:
def is_bad_sentence(string):
    bad_sentences = ["In Melbourne please happy", "npo way its in here qqqq"]
    for i in bad_sentences:
        if string==i:
            return False
    return True

In [6]:
indices = all_data["Tweets"].apply(less_than_8)
new_data = all_data.loc[indices].reset_index().drop("index", axis=1)
indices = new_data["Tweets"].apply(is_bad_sentence)
new_data = new_data.loc[indices].reset_index().drop("index", axis=1)
new_data

,Tweets,Score
0,An inspiration in all aspects: Fashion,1
1,fitness,1
2,beauty and personality. :)KISSES TheFashionIcon,1
3,Omg he... kissed... him crying with joy,1
4,thanks happy,1
...,...,...
1018,unhappy they not,0
1019,i miss so much unhappy,0
1020,Same unhappy .1,0
1021,Hi instant message your friend friend lang,0


In [7]:
random.seed(11)
shuffled = new_data.sample(frac=1)
shuffled.head()

,Tweets,Score
436,keep loving Deepika,1
665,Oo crying // Happy3YearsBraceUnited,0
854,Definitely my arms unhappy .13,0
531,Cleantha,0
169,Prepare to be inspired! happy,1


In [8]:
def score_to_array(num):
    return [num, 1-num]
shuffled["Score"] = shuffled["Score"].apply(score_to_array)
shuffled.head()

,Tweets,Score
436,keep loving Deepika,"[1, 0]"
665,Oo crying // Happy3YearsBraceUnited,"[0, 1]"
854,Definitely my arms unhappy .13,"[0, 1]"
531,Cleantha,"[0, 1]"
169,Prepare to be inspired! happy,"[1, 0]"


In [9]:
train_cutoff = int((0.8 * len(shuffled["Tweets"])) // 1)

In [10]:
twitter_training = shuffled.iloc[:train_cutoff].copy()
twitter_testing = shuffled.iloc[train_cutoff:].copy()

In [11]:
type(shuffled["Tweets"])

pandas.core.series.Series

In [12]:
train_data = list(twitter_training["Tweets"])

In [13]:
val_data = list(twitter_testing["Tweets"])

In [14]:
train_labels = list(twitter_training["Score"])
val_labels = list(twitter_testing["Score"])

In [15]:
val_labels = list(twitter_testing["Score"])

In [16]:
from lambeq import BobcatParser

parser = BobcatParser(root_cats=('NP', 'N'), verbose='text')

raw_train_diagrams = parser.sentences2diagrams(train_data, suppress_exceptions=True)
raw_val_diagrams = parser.sentences2diagrams(val_data, suppress_exceptions=True)

Tagging sentences.
Parsing tagged sentences.
Turning parse trees to diagrams.
Tagging sentences.
Parsing tagged sentences.
Turning parse trees to diagrams.


In [18]:
train_diagrams = [
    diagram.normal_form()
    for diagram in raw_train_diagrams if diagram is not None
]
val_diagrams = [
    diagram.normal_form()
    for diagram in raw_val_diagrams if diagram is not None
]

train_labels = [
    label for (diagram, label)
    in zip(raw_train_diagrams, train_labels)
    if diagram is not None]
val_labels = [
    label for (diagram, label)
    in zip(raw_val_diagrams, val_labels)
    if diagram is not None
]

In [19]:
len(train_diagrams)

625

In [20]:
len(train_labels)

625

In [21]:
from lambeq import AtomicType, IQPAnsatz, remove_cups
N = AtomicType.NOUN
S = AtomicType.SENTENCE
P = AtomicType.PREPOSITIONAL_PHRASE
PUNC = AtomicType.PUNCTUATION
C = AtomicType.CONJUNCTION

ansatz = IQPAnsatz({N: 1, S: 0, P:1},
                   n_layers=1, n_single_qubit_params=3)

train_circuits = []
trainlabels = []
for i in range(len(train_labels)):
    try:
        mycircuit = ansatz(remove_cups(train_diagrams[i]))
        train_circuits.append(mycircuit)
        trainlabels.append(train_labels[i])
    except ValueError:
        print(train_data[i])
print("END OF TRAINING SENTENCES")
#test_circuits = [ansatz(remove_cups(diagram)) for diagram in test_diagrams]
val_circuits = []
vallabels = []
for i in range(len(val_diagrams)):
    try:
        mycircuit = ansatz(remove_cups(val_diagrams[i]))
        val_circuits.append(mycircuit)
        vallabels.append(val_labels[i])
    except ValueError:
        print(val_data[i])

hawhaw crying  
miss you unhappy 
yes you had smile
 its texting without looking :)KISSES TheFashionIcon
Les videre happy
 thank you happy
77 ? Old Leyland
Yeaaaah! Alpha Invitation well received happy
 you'll be see.1
Hellooo.. happy  Lil strait (Daddyy_rat)
Noted on this Sir
Aww hating that I got sponsored
I'm so tired unhappy 
 DM us happy  S.1
hey.1
THANK YOU happy
END OF TRAINING SENTENCES
Update: I got in :D
You say VUCA
 Hitches Ride For Final unhappy  


In [22]:
from pytket.extensions.qiskit import AerBackend
from lambeq import TketModel

all_circuits = train_circuits + val_circuits

backend = AerBackend()
backend_config = {
    'backend': backend,
    'compilation': backend.default_compilation_pass(2),
    'shots': 8192
}

model = TketModel.from_diagrams(all_circuits, backend_config=backend_config)

In [23]:
from lambeq import BinaryCrossEntropyLoss

# Using the builtin binary cross-entropy error from lambeq
bce = BinaryCrossEntropyLoss()

acc = lambda y_hat, y: np.sum(np.round(y_hat) == y) / len(y) / 2  # half due to double-counting
eval_metrics = {"acc": acc}

In [24]:
from lambeq import QuantumTrainer, SPSAOptimizer

trainer = QuantumTrainer(
    model,
    loss_function=bce,
    epochs=150,
    optimizer=SPSAOptimizer,
    optim_hyperparams={'a': 0.05, 'c': 0.06, 'A':0.01*EPOCHS},
    evaluate_functions=eval_metrics,
    evaluate_on_train=True,
    verbose = 'text',
    seed=0
)

In [25]:
from lambeq import Dataset

train_dataset = Dataset(
            train_circuits,
            trainlabels,
            batch_size=BATCH_SIZE)

val_dataset = Dataset(val_circuits, vallabels, shuffle=False)

In [ ]:
trainer.fit(train_dataset, val_dataset, evaluation_step=1, logging_step=5)

In [ ]:
import matplotlib.pyplot as plt

fig, ((ax_tl, ax_tr), (ax_bl, ax_br)) = plt.subplots(2, 2, sharex=True, sharey='row', figsize=(10, 6))
ax_tl.set_title('Training set')
ax_tr.set_title('Development set')
ax_bl.set_xlabel('Iterations')
ax_br.set_xlabel('Iterations')
ax_bl.set_ylabel('Accuracy')
ax_tl.set_ylabel('Loss')

colours = iter(plt.rcParams['axes.prop_cycle'].by_key()['color'])
ax_tl.plot(trainer.train_epoch_costs[::10], color=next(colours))
ax_bl.plot(trainer.train_results['acc'][::10], color=next(colours))
ax_tr.plot(trainer.val_costs[::10], color=next(colours))
ax_br.plot(trainer.val_results['acc'][::10], color=next(colours))

# print test accuracy
test_acc = acc(model(val_circuits), val_labels)
print('Validation accuracy:', test_acc.item())